# Single Variable Analysis
---
### Notebook for Single Variable Analysis in all three processed datasets

Creation: 07.02.2021

## Required Libraries
---

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Importing Processed Datasets
---

In [20]:
leeds_accidents = pd.read_csv('../data/processed/accidents_processed.csv')
leeds_vehicles = pd.read_csv('../data/processed/vehicles_processed.csv')
leeds_casualties = pd.read_csv('../data/processed/casualties_processed.csv')

## Overview 
---
For each of the datasets, we want to get a first good impression of its size and the information it stores. To gain this information, we print out each of the datasets, and get a summary of each of the columns and the uniques.

### Accidents
---

In [21]:
leeds_accidents.shape # prints out the number of columns and rows

(1451, 32)

In [22]:
leeds_accidents # prints out an overview of the dataframe (and the number of rows and columns)

,Accident_Index,Location_Easting_OSGR,Location_Northing_OSGR,Longitude,Latitude,Police_Force,Accident_Severity,Number_of_Vehicles,Number_of_Casualties,Date,...,Pedestrian_Crossing-Human_Control,Pedestrian_Crossing-Physical_Facilities,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Did_Police_Officer_Attend_Scene_of_Accident,LSOA_of_Accident_Location
0,2019131901350,443576.0,438198.0,-1.339288,53.838231,13,2,4,1,27/09/2019,...,0,0,1,2,2,0,0,2,1,E01011309
1,20191358F1730,436147.0,434957.0,-1.452556,53.809670,13,3,2,7,15/08/2019,...,0,0,1,1,1,0,0,1,2,E01011666
2,2019136111190,435904.0,425850.0,-1.457300,53.727837,13,3,2,1,01/01/2019,...,0,0,1,1,1,0,0,2,1,E01011636
3,2019136111674,423194.0,438111.0,-1.649019,53.838752,13,3,1,1,01/01/2019,...,0,0,1,1,1,0,0,1,1,E01011461
4,2019136111836,429149.0,431736.0,-1.559127,53.781158,13,2,2,1,01/01/2019,...,0,0,4,1,1,0,0,1,1,E01011366
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1446,2019136CT0238,430040.0,434040.0,-1.545382,53.801815,13,2,1,1,29/12/2019,...,0,0,4,1,1,0,0,1,1,E01033010
1447,2019136CU0181,442094.0,434619.0,-1.362295,53.806186,13,3,1,1,30/12/2019,...,0,0,4,4,2,0,0,2,1,E01011297
1448,2019136CU0363,423019.0,437653.0,-1.651713,53.834643,13,2,1,1,30/12/2019,...,0,0,1,1,4,7,0,1,1,E01011452
1449,2019136CV0723,436853.0,442515.0,-1.440932,53.877548,13,2,2,1,31/12/2019,...,0,0,1,1,1,0,0,2,1,E01011713


In [23]:
leeds_accidents.nunique() # prints out the column names and the corresponding number of unique values 

Accident_Index                                 1451
Location_Easting_OSGR                          1353
Location_Northing_OSGR                         1319
Longitude                                      1407
Latitude                                       1404
Police_Force                                      1
Accident_Severity                                 3
Number_of_Vehicles                                6
Number_of_Casualties                              9
Date                                            348
Day_of_Week                                       7
Time                                            616
Local_Authority_(District)                        1
Local_Authority_(Highway)                         1
1st_Road_Class                                    6
1st_Road_Number                                  38
Road_Type                                         6
Speed_limit                                       6
Junction_Detail                                   9
Junction_Con

We see, that the main dataset 'accidents_processed.csv' stores all recorded accidents in 2019 in Leeds. It consist of 1451 columns (which leads to 1450 recorded accidents) and has 32 columns providing more detailed information about the accident. The different variables and the number of its unique values can be studied in the output of the above cell. We see, that we can differentiate the attributes as follows:
- Categorical Attributes (Most of the columns are categorical)
- Geographical Attributes (There are several measures of the location of the accident)
- Time Attribute (Each accident specifies a date and time)

### Vehicles
---

In [26]:
leeds_vehicles.shape # prints out the number of columns and rows

(2688, 23)

In [27]:
leeds_vehicles # prints out an overview of the dataframe (and the number of rows and columns)

,Accident_Index,Vehicle_Reference,Vehicle_Type,Towing_and_Articulation,Vehicle_Manoeuvre,Vehicle_Location-Restricted_Lane,Junction_Location,Skidding_and_Overturning,Hit_Object_in_Carriageway,Vehicle_Leaving_Carriageway,...,Journey_Purpose_of_Driver,Sex_of_Driver,Age_of_Driver,Age_Band_of_Driver,Engine_Capacity_(CC),Propulsion_Code,Age_of_Vehicle,Driver_IMD_Decile,Driver_Home_Area_Type,Vehicle_IMD_Decile
0,2019131901350,1,21,1,11,0,0,0,0,0,...,1,1,39,7,12419,2,3,1,1,1
1,2019131901350,2,21,0,4,0,0,0,0,0,...,1,1,56,9,6374,2,5,2,1,2
2,2019131901350,3,21,1,4,0,0,0,0,0,...,1,1,40,7,10837,2,2,4,1,4
3,2019131901350,4,21,1,4,0,0,0,0,0,...,1,1,35,6,-1,-1,-1,-1,-1,-1
4,20191358F1730,1,9,0,18,0,0,0,0,0,...,6,3,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2683,2019136CV0723,1,9,0,18,0,0,0,0,0,...,6,1,72,10,1968,2,4,10,1,10
2684,2019136CV0723,2,1,0,18,0,0,0,0,0,...,6,1,59,9,-1,-1,-1,10,1,10
2685,2019136CV1518,1,9,0,4,0,0,0,0,0,...,6,3,-1,-1,1598,2,7,7,1,7
2686,2019136CV1518,2,9,0,4,0,0,0,0,0,...,6,1,41,7,2183,2,7,3,1,3


In [28]:
leeds_vehicles.nunique() # prints out the column names and the corresponding number of unique values 

Accident_Index                      1451
Vehicle_Reference                      6
Vehicle_Type                          17
Towing_and_Articulation                5
Vehicle_Manoeuvre                     18
Vehicle_Location-Restricted_Lane       7
Junction_Location                      9
Skidding_and_Overturning               5
Hit_Object_in_Carriageway              9
Vehicle_Leaving_Carriageway            9
Hit_Object_off_Carriageway            11
1st_Point_of_Impact                    5
Was_Vehicle_Left_Hand_Drive?           2
Journey_Purpose_of_Driver              5
Sex_of_Driver                          3
Age_of_Driver                         84
Age_Band_of_Driver                    11
Engine_Capacity_(CC)                 229
Propulsion_Code                        6
Age_of_Vehicle                        30
Driver_IMD_Decile                     11
Driver_Home_Area_Type                  4
Vehicle_IMD_Decile                    11
dtype: int64

We see, that the side dataset 'vehicles_processed.csv' provides more detailed information about all vehicles involved in each of the accidents. It consist of 2688 columns (which leads to 2688 records on involved vehicles) and has 23 columns providing more detailed information about the vehicle. The different variables and the number of its unique values can be studied in the output of the above cell. We see, that we can differentiate the attributes as follows:
- Linking Attributes (Accident Indexes link the vehicles to the accidents dataset and the vehicle references the casualties)
- Categorical Attributes (Most of the columns are categorical)

### Casualties
---

In [30]:
leeds_casualties.shape # prints out the number of columns and rows

(1908, 16)

In [32]:
leeds_casualties # prints out an overview of the dataframe (and the number of rows and columns)

,Accident_Index,Vehicle_Reference,Casualty_Reference,Casualty_Class,Sex_of_Casualty,Age_of_Casualty,Age_Band_of_Casualty,Casualty_Severity,Pedestrian_Location,Pedestrian_Movement,Car_Passenger,Bus_or_Coach_Passenger,Pedestrian_Road_Maintenance_Worker,Casualty_Type,Casualty_Home_Area_Type,Casualty_IMD_Decile
0,2019131901350,1,1,1,1,39,7,2,0,0,0,0,0,21,1,1
1,20191358F1730,2,1,2,2,6,2,3,0,0,0,4,0,11,1,1
2,20191358F1730,2,2,2,1,9,2,3,0,0,0,4,0,11,1,1
3,20191358F1730,2,3,2,2,39,7,3,0,0,0,4,0,11,1,2
4,20191358F1730,2,4,2,1,5,1,3,0,0,0,4,0,11,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1903,2019136CU0181,1,1,1,1,27,6,3,0,0,0,0,0,9,2,9
1904,2019136CU0363,1,1,1,1,36,7,2,0,0,0,0,0,5,1,3
1905,2019136CV0723,2,1,1,1,59,9,2,0,0,0,0,0,1,1,10
1906,2019136CV1518,2,1,1,1,41,7,3,0,0,0,0,0,9,1,3


In [33]:
leeds_casualties.nunique() # prints out the column names and the corresponding number of unique values 

Accident_Index                        1451
Vehicle_Reference                        5
Casualty_Reference                      13
Casualty_Class                           3
Sex_of_Casualty                          2
Age_of_Casualty                         92
Age_Band_of_Casualty                    11
Casualty_Severity                        3
Pedestrian_Location                      9
Pedestrian_Movement                     10
Car_Passenger                            3
Bus_or_Coach_Passenger                   5
Pedestrian_Road_Maintenance_Worker       1
Casualty_Type                           18
Casualty_Home_Area_Type                  4
Casualty_IMD_Decile                     11
dtype: int64

We see, that the side dataset 'casualties_processed.csv' provides more detailed information about the casualties of all lethal accidents. It consist of 1908 columns (which leads to 1907 records on casualties) and has 16 columns providing more detailed information about the vehicle. The different variables and the number of its unique values can be studied in the output of the above cell. We see, that we can differentiate the attributes as follows:
- Linking Attributes (Accident Indexes link the vehicles to the accidents dataset and the vehicle references the casualties)
- Categorical Attributes (Most of the columns are categorical)

## Lookup and Variable Inspection
---

To efficiently analyse and plot singular attributes in the datasets, we need to keep track...
- ...of the column index of the column
- ...of the type of the variable (numerical, categorical)
- ...of the variable look-up for categorical data (those are taken from 'data/references/variable\ lookup.xls')

We therefore create a lookup dictionary for each of the datasets for a code efficient analysis.

### Accidents
---

In [36]:
# dictionary to keep track of relevant data in the accidents dataset
accidents_lookup = {
    0: {
        "Name": "Accident_Index", 
        "Type": "Categorical", 
        "Map": None
    },
    1: {
        "Name": "Location_Easting_OSGR",
        "Type": "Numerical", 
        "Map": None
    },
    2: {
        "Name": "Location_Northing_OSGR",
        "Type": "Numerical", 
        "Map": None
    }, 
    3: {
        "Name": "Longitude", 
        "Type": "Numerical",
        "Map": None
    },
    4: {
        "Name": "Latitude",
        "Type": "Numerical",
        "Map": None
    },
    5: {
        "Name": "Police_Force",
        "Type": "Categorical",
        "Map": {13: "West Yorkshire"}
    },
    6: {
        "Name": "Accident_Severity",
        "Type": "Categorical",
        "Map": {1: "Fatal", 2: "Serious", 3: "Slight"}
    }, 
    7: {
        "Name": "Number_of_Vehicles",
        "Type": "Categorical",
        "Map": None
    },
    8: {
        "Name": "Number_of_Casualties",
        "Type": "Categorical",
        "Map": None
    },
    9: {
        "Name": "Date",
        "Type": "Date",
        "Map": None
    },
    10: {
        "Name": "Day_of_Week",
        "Type": "Categorical",
        "Map": {1: "Monday", 2: "Tuesday", 3: "Wednesday", 4: "Thursday", 5: "Friday", 6: "Saturday", 7: "Sunday"}
    },
    11: {
        "Name": "Time",
        "Type": "Time",
        "Map": None
    },
    12: {
        "Name": "Local_Authority_(District)",
        "Type": "Categorical",
        "Map": {204: "Leeds"}
    },
    13: {
        "Name": "Local_Authority_(Highway)",
        "Type": "Categorical",
        "Map": {"E08000035": "Leeds"}
    },
    14: {
        "Name": "1st_Road_Class",
        "Type": "Categorical",
        "Map": {1: "Motorway", 2: "A(M)", 3: "A", 4: "B", 5: "C", 6: "Unclassified"}
    },
    15: {
        "Name": "1st_Road_Number",
        "Type": "Numerical",
        "Map": None
    },
    16: {
        "Name": "Road_Type",
        "Type": "Categorical",
        "Map": {1: "Roundabout", 2: "One-Way Street", 3: "Dual Carriageway", 6: "Single Carriageway", 7: "Slip Road", 
                9: "Unknown", 12: "One-Way Street/Slip Road", -1: "Data missing or out of range"}
    },
    17: {
        "Name": "Speed_limit",
        "Type": "Categorical",
        "Map": {20: "20 MPH", 30: "30 MPH", 40: "40 MPH", 60: "60 MPH", 70: "70 MPH", -1: "Data missing or out of range"}
    },
    18: {
        "Name": "Junction_Detail",
        "Type": "Categorical",
        "Map": {0: "Not at junction or within 20 metres", 1: "Roundabout", 2: "Mini-roundabout", 3: "T or staggered junction",                  5: "Slip road", 6: "Crossroads", 7: "More than 4 arms (not roundabout)", 8: "Private drive or entrance", 
                9: "Other junction", -1: "Data missing or out of range"}
    },
    19: {
        "Name": "Junction_Control",
        "Type": "Categorical",
        "Map": {0: "Not at junction or within 20 metres", 1: "Authorised person", 2: "Auto traffic signal", 3: "Stop sign", 
                4: "Give way or uncontrolled", 5: "Data missing or out of range"}
    },
    20: {
        "Name": "2nd_Road_Class",
        "Type": "Categorical",
        "Map": {1: "Motorway", 2: "A(M)", 3: "A", 4: "B", 5: "C", 6: "Unclassified"}
    },
    21: {
        "Name": "2nd_Road_Number",
        "Type": "Numerical",
        "Map": None
    },
    22: {
        "Name": "Pedestrian_Crossing-Human_Control",
        "Type": "Categorical",
        "Map": {0: "None within 50 metres", 1: "Control by school crossing patrol", 2: "Control by other authorised person",                    -1: "Data missing or out of range"}
    },
    23: {
        "Name": "Pedestrian_Crossing-Physical_Facilities",
        "Type": "Categorical",
        "Map": {0: "No physical crossing facilities within 50 metres", 1: "Zebra", 
                4: "Pelican, puffin, toucan or similar on-junction pedestrian light crossing", 
                5: "Pedestrian phase at traffic signal junction",7: "Footbridge or subway", 8: "Central refuge", 
                -1: "Data missing or out of range"}
    },
    24: {
        "Name": "Light_Conditions",
        "Type": "Categorical",
        "Map": {1: "Daylight", 4: "Darkness - lights lit", 5: "Darkness - lights unlit", 6: "Darkness - no lighting", 
                7: "Darkness - lighting unknown", -1: "Data missing or out of range"}
    },
    25: {
        "Name": "Weather_Conditions",
        "Type": "Categorical",
        "Map": {1: "Fine no high winds", 2: "Raining no high winds", 3: "Snowing no high winds", 4: "Fine + high winds", 
                5: "Raining + high winds", 6: "Snowing + high winds", 7: "Fog or mist", 8: "Other", 9: "Unknown", 
                -1: "Data missing or out of range"}
    },
    26: {
        "Name": "Road_Surface_Conditions",
        "Type": "Categorical",
        "Map": {1: "Dry", 2: "Wet or damp", 3: "Snow", 4: "Frost or ice", 5: "Flood over 3cm. deep", 6: "Oil or diesel", 7:                     "Mud", -1: "Data missing or out of range"}
    },
    27: {
        "Name": "Road_Surface_Conditions",
        "Type": "Categorical",
        "Map": {0: "None", 1: "Auto traffic signal - out", 2: "Auto signal part defective", 
                3: "Road sign or marking defective or obscured", 4: "Roadworks", 5: "Road surface defective", 
                6: "Oil or diesel", 7: "Mud", -1: "Data missing or out of range"}
    },
    28: {
        "Name": "Carriageway_Hazards",
        "Type": "Categorical",
        "Map": {0: "None", 1: "Vehicle load on road", 2: "Other object on road", 3: "Previous accident", 4: "Dog on road", 
                5:"Other animal on road", 6: "Pedestrian in carriageway - not injured", 
                7: "Any animal in carriageway (except ridden horse)", -1: "Data missing or out of range"}
    },
    29: {
        "Name": "Urban_or_Rural_Area",
        "Type": "Categorical",
        "Map": {1: "Urban", 2: "Rural", -1: "Unallocated"}
    },
    30: {
        "Name": "Did_Police_Officer_Attend_Scene_of_Accident",
        "Type": "Categorical",
        "Map": {1: "Yes", 2: "No", 3: "No - accident was reported using a self completion form (self rep only)"}
    },
    31: {
        "Name": "LSOA_of_Accident_Location",
        "Type": "Numerical",
        "Map": None
    }
}

### Vehicles
---

In [ ]:
# dictionary to keep track of relevant data in the vehicles dataset


### Casualties
---

In [ ]:
# dictionary to keep track of relevant data in the casualties dataset


## Single Variable Analysis
---

In [ ]:
def compute_summary_categorical(categorical_data):
    uniques, counts = np.unique(categorical_data, return_counts=True)

    return uniques, counts

In [ ]:
for column in range(leeds_accidents.shape[1]):
        if accidents_lookup[column]["Type"] == "Categorical":
            uniques, counts = compute_summary_categorical(leeds_accidents[accidents_lookup[column]["Name"]])
        
            accidents_lookup[column]["No_Uniques"] = len(uniques)
            if len(uniques) < 100:
                accidents_lookup[column]["Uniques"] = {uniques[i]: counts[i] for i in range(len(uniques))}

## Visualisation
---
In this section, we are visualising the results of the single variable analysis in all three datasets - both within the Jupyter and per export into 'reports/figures' for later use.

In [ ]:
def visualise_categorical(categorical_data):
    uniques, counts = np.unique(categorical_data, return_coutns=True)

    fig, axes = plt.subfigures()
    

In [ ]:
def print_overview(data):
    print("Column Index\t\tName\t\t\t\tType of Variable\t\t#Uniques\t\tUniques/ Counts")
    for i in range(len(data)):
        print(f"{i}\t\t\t{data[i].get('Name')}\t\t\t{data[i].get('Type')}\t\t{data[i].get('No_Uniques')}\t\t{data[i].get('Uniques')}")

print_overview(accidents_lookup)

### Categorical/ Numerical Values in Accidents

Column | Name | Variable Type | #Uniques | Uniques
--- | --- | --- | --- | ---
0 | Accident Index | Categorical | 1451  | Unique 13-digit indexes
1 | Location_Easting_OSGR | Numerical | 1353 | 6 digit number representing eastern location
2 | Location_Northing_OSGR | Numerical | 1319 | 6 digit number representing eastern location
3 | Longitude | Numerical | 1407 | Measure of Longitude
4 | Latitude | Numerical | 1404 |  Measure of Latitude
5 | Police_Force | Categorical | 1 | {13: "West Yorkshire"}
6 | Accident_Severity | Ordinal | 3 | {1: "Fatal", 2: "Serious", 3: "Slight"}
7 | Number_of_Vehicles | Numerical | 6 | Number of Vehicles involved
8 | Number_of_Casulaties | Numerical | 9 | Number of Casualties
9 | Date | Numerical | 348 | Dateformat
10 | Day_of_Week | Categorical | 7 | {1: "Monday", 2: "Tuesday", 3: "Wednesday", 4: "Thursday", 5: "Friday", 6: "Saturday", 7: "Sunday"}
11 | Time | Numerical | 616  | Times in 24H Format 
12 | Local_Authority_(District) | Categorical | 1 | {204: "Leeds"}
13 | Local_Authority_(Highway) | Categorical | 1| {E08000035: "Leeds"}
14 | 1st_Road_Class | Categorical | 7 | {1: "Motorway", 2: "A(M)", 3: "A", 4: "B", 5: "C", 6: "Unclassified"}
15 | 1st_Road_Number | Numerical | 38 | Identifier for Road
16 | Road_Type | Categorical | 6 | {1: "Roundabout", 2: "One way street", 3: "Dual carriageway", 6: "Single carriageway", 7: "Slip road", 9: "Unknown", 12: "One way street/Slip road", -1: "Data missing or out of range"}
17 | Speed_Limit | Numerical | 6 | {20, 30, 40, 60, 70, -1: "Data missing or out of range"}
18 | Junction_Detail | Categorical | 9 | {0: "Not at junction or within 20 metres", 1: "Roundabout", 2: "Mini-roundabout", 3: "T or staggered junction", 5: "Slip road", 6: "Crossroads", 7: "More than 4 arms (not roundabout)", 8: "Private drive or entrance", 9: "Other junction", -1: "Data missing or out of range"}
19 | Junction_Control | Categorical | 5 | {0: "Not at junction or within 20 metres", 1: "Authorised person", 2: "Auto traffic signal", 3: "Stop sign", 4: "Give way or uncontrolled", 5: "Data missing or out of range"}
20 | 2nd_Road_Class | Categorical | 7 | {1: "Motorway", 2: "A(M)", 3: "A", 4: "B", 5: "C", 6: "Unclassified"}
21 | 2nd_Road_Number | Numerical | 32 | Identifier for Road
22 | Pedestrian_Crossing-Human_Control | Categorical | 3 | {0: "None within 50 metres", 1: "Control by school crossing patrol", 2: "Control by other authorised person", -1: "Data missing or out of range"}
23 | Pedestrian_Crossing-Physical_Facilities | Categorical | 9 | {0: "No physical crossing facilities within 50 metres", 1: "Zebra", 4: "Pelican, puffin, toucan or similar non-junction pedestrian light crossing", 5: "Pedestrian phase at traffic signal junction",7: "Footbridge or subway", 8: "Central refuge", -1: "Data missing or out of range"}
24 | Light_Conditions | Categorical | 5 | {1: "Daylight", 4: "Darkness - lights lit", 5: "Darkness - lights unlit", 6: "Darkness - no lighting", 7: "Darkness - lighting unknown", -1: "Data missing or out of range"}
25 | Weather_Conditions | Categorical | 8 | {1: "Fine no high winds", 2: "Raining no high winds", 3: "Snowing no high winds", 4: "Fine + high winds", 5: "Raining + high winds", 6: "Snowing + high winds", 7: "Fog or mist", 8: "Other", 9: "Unknown", -1: "Data missing or out of range"}
26 | Road_Surface_Conditions | Categorical | 5 | {1: "Dry", 2: "Wet or damp", 3: "Snow", 4: "Frost or ice", 5: "Flood over 3cm. deep", 6: "Oil or diesel", 7: "Mud", -1: "Data missing or out of range"}
27 | Special_Conditions_at_Site | Categorical | 5 | {0: "None", 1: "Auto traffic signal - out", 2: "Auto signal part defective", 3: "Road sign or marking defective or obscured", 4: "Roadworks", 5: "Road surface defective", 6: "Oil or diesel", 7: "Mud", -1: "Data missing or out of range"}
28 | Carriageway_Hazards | Categorical | 6 | {0: "None", 1: "Vehicle load on road", 2: "Other object on road", 3: "Previous accident", 4: "Dog on road", 5: "Other animal on road", 6: "Pedestrian in carriageway - not injured", 7: "Any animal in carriageway (except ridden horse)", -1: "Data missing or out of range"}
29 | Urban_or_Rural_Area | Categorical | 3 | {1: "Urban", 2: "Rural", -1: "Unallocated"}
30 | Did_Police_Officer_Attend_Scene_of_Accident | Categorical | 2 | {1: "Yes", 2: "No", 3: "No - accident was reported using a self completion  form (self rep only)"}
31 | LSOA_of_Accident_Location | Numerical | 379 | A Lower Layer Super Output Area is a GEOGRAPHIC AREA. Lower Layer Super Output Areas (LSOA) are a geographic hierarchy designed to improve the reporting of small area statistics in England and Wales